## Baseball's Steroid Era

### Using machine learning to find out who else was on the juice

#### Chris Johnson - January 2018

When I was a kid I collected baseball cards.  My favorite team was the Oakland A's.  In the late 1980's, the best part about the A's was the Bash Brothers, Jose Canseco and Mark McGwire.  They were my baseball heroes.  It turns out they were also pretty regular steroid users.  We know from the Mitchell report that more than 80 other players were juicing too.  I want to make a model that can predict which other sluggers may also have been using performance enhancing drugs based only on baseball performance stats.

I'll use the The Lahman Baseball Database 2014 version (http://www.seanlahman.com/baseball-archive/statistics/) as the basis for this analysis.  My idea is to take the top sluggers in baseball history and split them into three categories.

Category 1: Sluggers who played before the steroid era that I'll assume were clean

Category 2: Sluggers who have been implicated as involved with performance enhancing drugs

Category 3: Sluggers who had retired before 2014, played during the steroid era, and weren't implicated as involved with performance enhancing drugs

Categories 1 & 2 will train and test a binary classifier.  I'll then make predictions using the Category 3 data.   

### Get Some Data

My first task is to get a dataframe with all the top sluggers going back to Ruth (all time leader in career slugging percentage).  To determine sluggers I'll add a slugging percentage column to the dataframe, put it in order of descending slugging percentage, and keep top 1500 or so sluggers.  According to Wikipedia, slugging percentage (SLG) is defined as:

$$
SLG = \frac{(1B + 2(2B) + 3(3B) + 4(HR))}{AB}
$$

Loosely following the lead of baseball-reference.com, I'll only include batters that have a minimum of:
    
    1) 3000 at bats and
    2) 500 career games played
   


In [81]:
import sqlite3
import pandas
import numpy as np

sqlite_file = 'lahman2014.sqlite'
conn = sqlite3.connect(sqlite_file)
#get all players with more than 3000 AB's and 500 games. also bring back info to calculate SLG
Slugger_query = "SELECT playerID as TheKey, sum(G) as GamesPlayed, sum(AB) as AtBats, sum(HR) as HomeRuns, sum(H) as Hits, sum([3B]) as Triples, sum([2B]) as Doubles FROM Batting GROUP BY playerID HAVING sum(G) >= 500 AND sum(AB) >= 3000"
#make a dataframe
Sluggers = pandas.read_sql(Slugger_query, conn)
#add column for singles
Sluggers['Singles'] = Sluggers['Hits'] - Sluggers['Triples'] - Sluggers['Doubles'] - Sluggers['HomeRuns']
#add column for SLG
Sluggers['Slugging'] = (Sluggers['Singles'] + (2*Sluggers['Doubles']) + (3*Sluggers['Triples']) + (4*Sluggers['HomeRuns']))/Sluggers['AtBats']
#just keep 1500 best sluggers
DescendingSluggers = Sluggers.sort_values(by='Slugging',ascending=False)
Best1500Sluggers = DescendingSluggers[:1500]

### Add More Information

I've got my list of the top sluggers from all baseball eras.  To better define the player's baseball era, I'll put in the start date and retirement date of each of these players.  Then I'll identify whether they retired before the steroid era.  I should toss their names in too.  To help control for changes in the early game, I'll only get data from after baseball's deadball era.  I'll date the end of the deadball era to the year that Babe Ruth started hitting more or less full time, 1919.  For later age based trends, I'll add in player birth year as well.

In espn's estimation, the steroids era in baseball began in the late 1980's.  Jose Canseco was a late call up in 1985.  Mark Mcgwire, Barry Bonds, and Rafael Palmeiro made their major league debuts in 1986.  These four are part of the same baseball generation and seem to mark the arrival of a new kind of 'competitor'.  But, let's give the old guys a couple years to clear out.  I'll put the steroids era cutoff date at 1988.

I'll also add in info from http://www.baseballssteroidera.com/bse-list-steroid-hgh-users-baseball.html that identifies players linked to steroids or human growth hormone.


In [82]:
#get names and dates
SluggerInfo_query = "SELECT playerID as TheKey, nameFirst, nameLast, debut, finalGame, birthYear FROM MASTER"
SluggersInfo = pandas.read_sql(SluggerInfo_query, conn)
#do an inner join on the two df's
#this only keeps records that appear in both dataframes
TheSluggersDf = pandas.merge(Best1500Sluggers, SluggersInfo, how='inner',left_on='TheKey', right_on='TheKey')

#convert date columns to date data types
#sqllite has number of ms since 1JAN1970, so have to adjust in the following way
TheSluggersDf['debut'] = pandas.to_timedelta(TheSluggersDf['debut'], unit='ms') + pandas.Timestamp('1970-1-1')
TheSluggersDf['finalGame'] = pandas.to_timedelta(TheSluggersDf['finalGame'], unit='ms') + pandas.Timestamp('1970-1-1')

#many players on this list are still active.  the meta states that finalGame is blank for active players, but it
#active players seem to return a date from 2014.  I'll just include an indicator for all players with
#a finalGame in 2014 (1 is active, -1 is retired)
TheSluggersDf['Active'] = np.where(TheSluggersDf['finalGame'].dt.year == 2014, 1,-1)

#add boolean indicating whether player retired before the steroids era or not (1 is true, -1 is false)
TheSteroidCutoff = np.datetime64('1988-04-04')
TheSluggersDf['SteroidsEra'] = np.where(TheSluggersDf['finalGame']>=TheSteroidCutoff, 1, -1)

#exclude deadball era players
TheDeadballCutoff = np.datetime64('1919-04-19')
NonDeadballSluggersDf = TheSluggersDf[TheSluggersDf['finalGame'] >= TheDeadballCutoff]
print("after applying the deadball filter we have " + str(NonDeadballSluggersDf.shape[0]) + " slugger's stats")

#add in info about players linked to steroids and hgh
UsersDf = pandas.read_csv('Users.csv')
#do a left join on the two df's
#this only keeps all records from NonDeadballSluggersDf and only adds info from matches in UsersDf
TheSluggersUsersDf = pandas.merge(NonDeadballSluggersDf, UsersDf, how='left',left_on=['nameFirst','nameLast'], right_on=['fName','lName'])
#drop new fName,lName columns
TheCleanerSluggersDf = TheSluggersUsersDf.drop(['fName','lName'], axis=1)
print(str(NonDeadballSluggersDf.shape[0] - TheCleanerSluggersDf['status'].isnull().sum()) + " are linked to steroids or hgh according to baseballssteroidera.com")
#play the 1,-1 game on status.  1 means a link to steroids or hgh
TheCleanerSluggersDf['status'] = np.where(TheCleanerSluggersDf['status'].isnull(), -1, 1)
#TheCleanerSluggersDf.head()

#make a dataframe for just retired players
RetiredSluggers = TheCleanerSluggersDf[TheCleanerSluggersDf['Active'] < 0]

print("after applying the retired filter we have " + str(RetiredSluggers.shape[0]) + " slugger's stats")
LinkedSluggers = RetiredSluggers[RetiredSluggers['status'] > 0]
print(str(LinkedSluggers.shape[0]) + " retired players are linked to steroids or hgh")

after applying the deadball filter we have 1308 slugger's stats
42 are linked to steroids or hgh according to baseballssteroidera.com
after applying the retired filter we have 1178 slugger's stats
39 retired players are linked to steroids or hgh


### Explore the Data

It's time to look for features that might distinguish between players who to took steroids and players who did not.

Although it's not a sure assumption that all sluggers who played in the pre-steroid era were clean, it's the one I'm going with here.  There are some striking differences in achievements between the eras.  There is a pretty good story on the subject from Bleacher Report (http://bleacherreport.com/articles/1648362-proof-that-the-steroid-era-power-surge-in-baseball-has-been-stopped).

Some quotes from the report:

"The Steroid Era saw an explosion of 40-homer seasons, which have since gone back to being special occasions...42 percent of MLB's 40-homer seasons happened" between 1996 and 2006.

"Old guys were doing things in the juiced-up era that old guys generally have no business doing.  Case in point, there have been 24 cases in MLB history of a player 35 years old or older hitting at least 40 HRs. Of those, 13 occurred between 1996 and 2006."

ESPN has a good summary of steroid era facts too (http://www.espn.com/mlb/topics/_/page/the-steroids-era). 

"While just three players reached the 50-home run mark in any season between 1961 and 1994, many sluggers would start to surpass that number in the mid-90s."

"Sosa finished second in the NL in home runs with 66, 26 more than his previous season high."

"Bonds notched 73 homers despite failing to reach the 50-home run plateau in any prior season."

It seems year over year home run total differences and late career performance improvements are potential areas of interest to explore.

I'll ask five questions of the data.

Q1: Did this player hit 40 homeruns in a season at 35 years of age or more?

Q2: Did this player ever have a season (for >= 400 AB's) that his HR total change was>=15 HR from previous season?

Q3: Did this player ever have a season (for >= 400 AB's) that his HR total change was>=20 HR from previous season?

Q4: Is this players mean over 35 SLG better than his mean SLG from the age 27-30?

Q5: Is this players mean over 35 HR per AB better than his mean HR per AB from the age 27-30?



In [153]:
#write a function to ask a series of questions for each player:

#1) did this player hit 40 homeruns in a season at 35 years of age or more?
#2 and 3) did this player ever have a season (for >= 400 AB's) that his HR total change was>=15 HR from previous season? >= 20?
# for q's 2 and 3 need to control for rookies.  make sure players are mature before testing.  say 28 or older
#read this if you think this is a bad threshold
#https://www.baseballprospectus.com/news/article/9933/how-do-baseball-players-age-investigating-the-age-27-theory/
#4) is this players mean over 35 SLG better than his mean SLG from the age 27-30?
#5) is this players mean over 35 HR per AB better than his mean HR per AB from the age 27-30?

#The input to the function is a series of player ids and a corresponsing series of birth years

def GetSomeAnswers(ThePlayerInfo):
    #an empty dataframe to keep all the answers in
    DfColumns=['TheKey','Q1','Q2','Q3','Q4','Q5']
    DfList = []
    for row in ThePlayerInfo.itertuples():
        ThePlayerId = row.TheKey
        TheBirthYear = row.birthYear
        
        SluggerTs_query = "SELECT yearID as TheKey, AB, HR, H, [3B], [2B] FROM Batting WHERE playerID = '" + ThePlayerId + "' GROUP BY yearID HAVING AB >= 400"
        SluggersTs = pandas.read_sql(SluggerTs_query, conn)
        #add column for singles
        SluggersTs['Singles'] = SluggersTs['H'] - SluggersTs['3B'] - SluggersTs['2B'] - SluggersTs['HR']
        #add column for SLG
        SluggersTs['Slugging'] = (SluggersTs['Singles'] + (2*SluggersTs['2B']) + (3*SluggersTs['3B']) + (4*SluggersTs['HR']))/SluggersTs['AB']
        #add column for age
        SluggersTs['Age'] = SluggersTs['TheKey'] - int(TheBirthYear)
        #add column for HR change
        SluggersTs['Subtractor'] = SluggersTs['HR'].shift(periods=1)
        SluggersTs['HrChange'] = SluggersTs['HR'] - SluggersTs['Subtractor']
        CleanSluggersTs = SluggersTs.drop(['Subtractor'], axis=1)
        #add column for HR/AB
        CleanSluggersTs['HrRate'] = SluggersTs['HR']/SluggersTs['AB']
        
        #answer Q1 did this player hit 40 homeruns in a season at 35 years of age or more?
        Over35HrDf = CleanSluggersTs[CleanSluggersTs['Age'] >= 35].HR
        Answer1 = (Over35HrDf >= 40).any()
        if Answer1:
            Over35Hr = 1
        else:
            Over35Hr = -1
        #print(Over35Hr,Answer1)
        #answer Q2 did this player ever have a seasonthat he hit more/less than 15 HR than the previous season?
        GtPos15Df = CleanSluggersTs[CleanSluggersTs['Age'] >= 28].HrChange.abs()
        Answer2 = (GtPos15Df >= 15).any()
        if Answer2:
            GtPos15 = 1
        else:
            GtPos15 = -1
        #print(GtPos15,Answer2)
        #answer Q3 did this player ever have a seasonthat he hit more/less than 20 HR than the previous season?
        GtPos20Df = CleanSluggersTs[CleanSluggersTs['Age'] >= 28].HrChange.abs()
        Answer3 = (GtPos20Df >= 20).any()
        if Answer3:
            GtPos20 = 1
        else:
            GtPos20 = -1
        #print(GtPos20,Answer3)
        #answer Q4 is this players mean over 35 SLG better than his mean SLG from the age 27-30?
        Slg35 = CleanSluggersTs[CleanSluggersTs['Age'] >= 35].Slugging.mean()
        SlgPrime = CleanSluggersTs[(CleanSluggersTs['Age'] >= 27) & (CleanSluggersTs['Age'] <= 30)].Slugging.mean()
        #print("slg means",[Slg35,SlgPrime])
        Answer4 = Slg35 > SlgPrime
        if Answer4:
            SlgTest = 1
        else:
            SlgTest = -1
        #print(SlgTest,Answer4)
        #answer Q5 is this players mean over 35 HR per AB better than his mean HR per AB from the age 27-30?
        HrAb35 = CleanSluggersTs[CleanSluggersTs['Age'] >= 35].HrRate.mean()
        HrAbPrime = CleanSluggersTs[(CleanSluggersTs['Age'] >= 27) & (CleanSluggersTs['Age'] <= 30)].HrRate.mean()
        #print("HrAb",[HrAb35,HrAbPrime])
        Answer5 = HrAb35 > HrAbPrime
        if Answer5:
            HrAbTest = 1
        else:
            HrAbTest = -1
        #print(HrAbTest,Answer5)
        
        DfList.append([ThePlayerId,Over35Hr,GtPos15,GtPos20,SlgTest,HrAbTest])
    
    SomeAnswers = pandas.DataFrame(DfList,columns=DfColumns)
    return(SomeAnswers)
        


In [255]:
#input to the function are all the RetiredSluggers
MyPlayerInfo = pandas.concat([RetiredSluggers['TheKey'], RetiredSluggers['birthYear']], axis=1)

TheAnswers = GetSomeAnswers(MyPlayerInfo)
#TheAnswers.head()

#then i'll add the answers (-1 no, 1 yes) to the retired sluggers df
#do a left join on the two df's
#this only keeps all records from RetiredSluggers and only adds info from matches in TheAnswers
RetiredSluggersAnswers = pandas.merge(RetiredSluggers, TheAnswers, how='left',left_on=['TheKey'], right_on=['TheKey'])
#RetiredSluggersAnswers.head()

### Prepare Data for the Classifier

At this point there are some features in the dataset I can use to train a model.  I'll break the data into three parts.  There are people I'm assuming didn't use steroids or hgh (SteroidsEra = -1).  There is a list of people linked to steroids or hgh (status = 1). There are also some who played during the steroids era that aren't linked to steroids or hgh (SteroidsEra*status < 0).  That last group will be the group i'll make the predictions on. The rest I'll use as a train and test set.

In [257]:
#add indicator for train/test or predictor and make the groups
RetiredSluggersAnswers['PredictionGroup'] = RetiredSluggersAnswers['SteroidsEra']*RetiredSluggersAnswers['status']
PredictionGroup = RetiredSluggersAnswers[RetiredSluggersAnswers['PredictionGroup']< 0]
#print("prediction group",PredictionGroup.shape)
LinkedPredictionSluggers = PredictionGroup[PredictionGroup['status'] > 0]
#print(str(LinkedPredictionSluggers.shape[0]) + " prediction group players are linked to steroids or hgh")
#print(LinkedPredictionSluggers)
TrainTestGroup = RetiredSluggersAnswers[RetiredSluggersAnswers['PredictionGroup']> 0]
print("train test group",TrainTestGroup.shape)
LinkedTrainTestSluggers = TrainTestGroup[TrainTestGroup['status'] > 0]
print(str(LinkedTrainTestSluggers.shape[0]) + " train/test players are linked to steroids or hgh")

train test group (722, 23)
38 train/test players are linked to steroids or hgh


Now i'll split the train/test data into train and test groups keeping approximately equal ratios of confirmed users in each group.

In [219]:
#make train and test datasets with approximately equal ratios of confirmed users
#import sklearn shuffle split
from sklearn.model_selection import StratifiedShuffleSplit

#make a unique index for each entry with nice ordered integers
TrainTestGroupDf = TrainTestGroup.reset_index(drop=True)
#just keep the fields to use for training
TrainTestGroupCopy = TrainTestGroupDf[['Q1','Q2','Q3','Q4','Q5','status']].copy()

#figure out the frequency of each score for the whole data set
print('All Data')
print(TrainTestGroupCopy['status'].value_counts() / len(TrainTestGroupCopy['status']))

#do stratified sampling
#see: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
split = StratifiedShuffleSplit(n_splits=1, test_size=200, random_state=1234)
for train_index, test_index in split.split(TrainTestGroupCopy, TrainTestGroupCopy["status"]):
    train_set = TrainTestGroupCopy.loc[train_index]
    test_set = TrainTestGroupCopy.loc[test_index]
    
#compare test and training data sets to see that ratios approximately match
print('Training Data')
print(train_set["status"].value_counts() / len(train_set))
print('Test Data')
print(test_set["status"].value_counts() / len(test_set))
#print(train_set)
#set up x and y for the train set
train_X = train_set[['Q1','Q2','Q3','Q4','Q5']]
train_y = train_set[['status']]
#reshape the labels to make this work
labels = train_y['status']
#set up x and y for the test set
test_X = test_set[['Q1','Q2','Q3','Q4','Q5']]
test_y = test_set[['status']]
#reshape the labels to make this work
labelsTest = test_y['status']
#print(train_y.shape)
#print(train_X.shape)

All Data
-1    0.947368
 1    0.052632
Name: status, dtype: float64
Training Data
-1    0.948276
 1    0.051724
Name: status, dtype: float64
Test Data
-1    0.945
 1    0.055
Name: status, dtype: float64


### Model Selection/Evaluation

Here I'll try out an ensemble method, Adaptive Boosting (Adaboost).  In general, ensemble methods take a combination of weak learners in order to make a stronger learner.  Adaboost iterates through a classifier making predictions for each iteration.  If a predecessor classifier is wrong, then going forward it puts more weight on the incorrect predictions.  I'm using decistion trees as the weak learners here.  

To evaluate this model I'll use cross validation scores, in this case the area under the ROC curve.  I'll use 20 folds during cross validation.

In [242]:
#get cross validation scores (small dataset here) for the adaboost classifier using 
#decision tree stumps as the weak learners

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import numpy as np

#Use decision trees as weak learners
TreeStump = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)

clf = AdaBoostClassifier(base_estimator=TreeStump,n_estimators=100)
#specify 20 fold
scores = cross_val_score(clf, train_X, labels,scoring='roc_auc',cv=20)
print('All done. Here are the auc scores: ',scores)
print('And the mean auc score (mean estimate of classification accuracy): ',np.mean(scores))


All done. Here are the auc scores:  [ 0.9         0.74        0.71        0.98        0.42        0.5         0.87
  0.38        1.          0.98        0.98        0.44        0.44        0.96
  0.96        0.85416667  0.4375      0.39583333  1.          0.85416667]
And the mean auc score (mean estimate of classification accuracy):  0.740083333333


The results from cross validation indicate this approach has potential.  I'll train a model and get some metrics about how it behaves on the test data.

In [244]:
#use adaboost then make predictions on the test data
Ada = clf.fit(train_X, labels)
yPred = Ada.predict(test_X)
target_names = ['clean', 'dirty']

from sklearn.metrics import classification_report
print(classification_report(labelsTest, yPred, target_names=target_names))





             precision    recall  f1-score   support

      clean       0.95      1.00      0.97       189
      dirty       1.00      0.09      0.17        11

avg / total       0.95      0.95      0.93       200



The accuracy of the positive predictions, or precision, is high for both clean and dirty players.  That means when the model actually calls someone out for using, it's usually right.  But, the ratio of positive instances correctly detected, or recall, is pretty low for dirty players.  That means it's a pretty conservative classifier with respect to going out on a limb to identify those who used performance enhancing drugs.

### Predict who else used Performance Enhancing Drugs

This is the part where I name names.

In [254]:
#now see if any of the other guys from the prediction group are dirty

#make a unique index for each entry with nice ordered integers
PredictionGroupDf = PredictionGroup.reset_index(drop=True)
#just keep the fields to use for training
PredictionGroupCopy = PredictionGroupDf[['Q1','Q2','Q3','Q4','Q5','status']].copy()
#set up x and y for the prediction set
Pg_X = PredictionGroupCopy[['Q1','Q2','Q3','Q4','Q5']]
#see who may have taken steroids or hgh
PgPred = Ada.predict(Pg_X)
#add the predictions to the PredictionGroupDf
PredictionGroupDf['Predictions'] = PgPred
NonDeadballSluggersDf = TheSluggersDf[TheSluggersDf['finalGame'] >= TheDeadballCutoff]
PotentialUsers = PredictionGroupDf[PredictionGroupDf['Predictions'] > 0]
PotentialUsersNames = PotentialUsers[['nameFirst', 'nameLast']]

print("according to my model, these guys may also have been using steroids or hgh")
PotentialUsersNames



according to my model, these guys may also have been using steroids or hgh


,nameFirst,nameLast
0,Larry,Walker
11,Mike,Schmidt
17,Ellis,Burks
58,Luis,Gonzalez
79,Greg,Vaughn
153,Steve,Finley
193,Darrell,Evans
233,Terry,Steinbach
250,Jay,Bell


The first name that jumps out to me is Terry Steinbach.  He was on those Bash Brother teams in Oakland.  According to a blog (http://joeposnanski.com/32-flukiest-home-run-seasons/), he had the 18th flukiest home run season ever in 1996.  My model didn't know he played with Canseco and McGwire, but it saw that he hit way more homers in '96 than in any other season.  It's tough to explain this aberration, but one writer has not ruled out some sort of deal with Lucifer (http://twinsdaily.com/blog/65/entry-6196-steinbach-in-96-how-do-you-explain-it/).

Larry Walker and Ellis Burks had some monster seasons in '96 and '97.  They are on some people's radar (http://bleacherreport.com/articles/171276-can-anyone-be-trusted-looking-at-the-steroid-era, http://bleacherreport.com/articles/1011219-most-suspicious-one-year-position-player-wonders-in-mlb-history).  Again, I think it was the home run total jump that my model saw.  Also they played for the Rockies during that time frame with Walt Weiss, another Bash Brother teammate from Oakland.  Of course, home run totals in the thin air of Coors Field are bigger than in other parks...

Mike Schmidt supposedly used amphetamines (https://nyulocal.com/juicin-in-the-majors-a-history-of-steroids-in-baseball-d2facd1cbcfb), but I'm not convinced he used steroids.  He is from a baseball generation that I tried to exclude from consideration, but he didn't retire until 1989, so he got looked at here.  I think the model saw his rough offensive year in '78 and drew some questionable conclusions.  The thing is, and this is in contrast especially to Steinbach and Burks, Schmidt's bad year in '78 was the aberration.  He didn't just have one amazing year.